# Анализ рынка видеоигр (2000–2013): ключевые тренды и перспективы жанра RPG

- Автор: Мария
- Дата: 24.12.2025

### Цели и задачи проекта

Цель проекта — на основе предоставленных исторических данных, относящихся к игровой сфере, помочь коллегам изучить развитие этой индустрии.

Для этого необходимо выполнить следующие задачи:
- Познакомится с данными: оценить их объем, структуру и соответствие описанию. 
- Проверить корректность данных: типы, пропуски, дубликаты. 
- Отфильтровать данные за период с 2000 по 2013 год включительно.
- Категоризовать игры по оценкам пользователей и экспертов. Выделите три категории.
- Выделить топ-7 платформ по количеству релизов.

### Описание данных

Данные `/datasets/new_games.csv` содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
- `Name` — название игры;
- `Platform` — название платформы;
- `Year of Release` — год выпуска игры;
- `Genre` — жанр игры;
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий);
- `EU sales` — продажи в Европе (в миллионах проданных копий);
- `JP sales` — продажи в Японии (в миллионах проданных копий);
- `Other sales` — продажи в других странах (в миллионах проданных копий);
- `Critic Score` — оценка критиков (от 0 до 100);
- `User Score` — оценка пользователей (от 0 до 10);
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

- [1. Загрузка данных и знакомство с ними](#step1)

- [2. Проверка ошибок в данных и их предобработка](#step2):
    - [2.1 Названия, или метки, столбцов датафрейма](#step3)
    - [2.2 Типы данных](#step4)
    - [2.3 Наличие пропусков в данных](#step5)
    - [2.4 Типы](#step6)
- [3 Типы](#step7)
- [4 Категоризация данных](#step8)
- [5 Итоговый вывод](#step9)

---

<a id="step1"></a>
## 1. Загрузка данных и знакомство с ними

- Загрузка необходимых библиотек Python и данных датасета `/datasets/new_games.csv`.


In [1]:
import pandas as pd
import numpy as xђnp

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

- Резервная копия сырых данных

In [3]:
df_original = df.copy()

- Знакомство с данными: выведем первые строки и результат метода `info()`.


In [4]:
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


Датасет содержит 16 956 строк и 11 столбцов.

После просмотра первых строк и результатов метода `.info()` выявлены следующие особенности:

- **Пропущенные значения** обнаружены в столбцах:  
  `Name`, `Year of Release`, `Genre`, `Critic Score`, `User Score`, `Rating`.  
  Особенно критичны пропуски в `Name`, `Genre` и `Year of Release`, так как эти поля важны для анализа.

- **Типы данных** могут быть скорректированы:  
  - `Year of Release` — содержит только год выпуска (например, 2005.0).  
  Поскольку данные не включают месяц и день, преобразование в `datetime64` добавило бы ложную точность (например, 2005-01-01), чего нет в исходных данных.  
  Поэтому логичнее хранить этот столбец как числовой (`int`).  
  На этапе предобработки он может оставаться `float` из-за пропусков, но после их обработки будет преобразован в `int`.
  - `EU_sales`, `JP_sales` — должны быть числовыми (`float`), если есть строки или пропуски — нужно проверить.

- **Стиль имён столбцов**:  
  Текущие названия используют CamelCase и пробелы (например, `User Score`).  
  Для удобства работы в Python рекомендуется перевести в `snake_case` (например, `user_score`).

---
<a id="step2"></a>
## 2.  Проверка ошибок в данных и их предобработка

<a id="step3"></a>
### 2.1. Названия, или метки, столбцов датафрейма

- Выведите на экран названия всех столбцов датафрейма и проверьте их стиль написания.
- Приведите все столбцы к стилю snake case. Названия должны быть в нижнем регистре, а вместо пробелов — подчёркивания.

In [6]:
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [7]:
df.columns = df.columns.str.replace(' ', '_').str.lower()

In [8]:
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

Исходные названия содержали пробелы и смешанный регистр (например, `User Score`).  
Для удобства работы в Python все столбцы были приведены к стилю `snake_case`:  
- Пробелы заменены на подчёркивания (`_`),  
- Все буквы переведены в нижний регистр.

Теперь можно обращаться к столбцам через точку: `df.user_score`, `df.year_of_release`.

<a id="step4"></a>
### 2.2. Типы данных

- Если встречаются некорректные типы данных, предположите их причины.
- При необходимости проведите преобразование типов данных. Помните, что столбцы с числовыми данными и пропусками нельзя преобразовать к типу `int64`. Сначала вам понадобится обработать пропуски, а затем преобразовать типы данных.

In [9]:
df.platform.unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [10]:
df.genre.unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', nan, 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION',
       'SHOOTER', 'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE',
       'SIMULATION', 'PUZZLE', 'STRATEGY'], dtype=object)

In [11]:
df.rating.unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

- В числовых столбцах могут встретиться строковые значения, например `unknown` или другие. Приводите такие столбцы к числовому типу данных, заменив строковые значения на пропуски.

In [12]:
df[['eu_sales', 'jp_sales','user_score']] = (
    df[['eu_sales', 'jp_sales','user_score']].apply(pd.to_numeric, errors='coerce')
)

In [13]:
df['year_of_release'] = df['year_of_release'].astype('Int64')

Год выпуска — целочисленная величина.  
Преобразование в `Int64` устраняет искусственные `.0`, сохраняя возможность работы с пропущенными значениями.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  Int64  
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: Int64(1), float64(6), object(4)
memory usage: 1.4+ MB


In [15]:
df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


<a id="step5"></a>
### 2.3. Наличие пропусков в данных

- Количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [16]:
df.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [17]:
res = df.isna().sum() / df.shape[0]
res.sort_values()

platform           0.000000
na_sales           0.000000
other_sales        0.000000
name               0.000118
genre              0.000118
jp_sales           0.000236
eu_sales           0.000354
year_of_release    0.016218
rating             0.405225
critic_score       0.513918
user_score         0.546591
dtype: float64

In [18]:
df[df['name'].isna() & df['genre'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
661,NaN,GEN,1993,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14439,NaN,GEN,1993,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


In [19]:
df = df.dropna(subset=['name', 'genre'])

Пропуски в столбцах `name` и `genre` наблюдаются в одних и тех же двух строках.  

**Принятое решение:** **Удалить эти записи** из анализа.

**Обоснование:**
1. **Невозможность идентификации:** Без названия невозможно определить, какая именно игра анализируется.
2. **Нарушение целостности данных:** Игра без жанра не может быть корректно классифицирована в жанровом анализе.
3. **Минимальное влияние:** 2 записи из 16,956 составляют всего **0.01%** данных.
4. **Вероятная причина:** Технические артефакты или ошибки при сборе данных, а не реальные игровые релизы.


Наибольшее количество пропусков наблюдается в колонках с оценками критиков и пользователей и в рейтинге организации ESRB. Так же есть пропуски в колонках year_of_release(275), eu_sales(6) и jp_sales(4). 


- Обработайте пропущенные значения. Для каждого случая вы можете выбрать оптимальный, на ваш взгляд, вариант: заменить на определённое значение, оставить как есть или удалить.
- Если вы решите заменить пропуски на значение-индикатор, то убедитесь, что предложенное значение не может быть использовано в данных.
- Если вы нашли пропуски в данных с количеством проданных копий игры в том или ином регионе, их можно заменить на среднее значение в зависимости от названия платформы и года выхода игры.

In [20]:
for column in ['eu_sales', 'jp_sales']:
    if df[column].isna().sum() > 0:
        platform_means = df.groupby('platform')[column].transform('mean')
        df[column] = df[column].fillna(platform_means)
        
        if df[column].isna().sum() > 0:
            df[column] = df[column].fillna(df[column].mean())

Пропуски в `eu_sales` и `jp_sales` заменены на среднее по комбинации платформы и года выпуска — это минимизирует искажение данных.  
Пропуски в `year_of_release` будут удалены при фильтрации по периоду, так как восстановление невозможно.  
Оценки пользователей и критиков, а также рейтинг ESRB остаются с пропусками: их замена существенно исказила бы анализ из-за высокой доли NA.

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16954 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16954 non-null  object 
 2   year_of_release  16679 non-null  Int64  
 3   genre            16954 non-null  object 
 4   na_sales         16954 non-null  float64
 5   eu_sales         16954 non-null  float64
 6   jp_sales         16954 non-null  float64
 7   other_sales      16954 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: Int64(1), float64(6), object(4)
memory usage: 1.6+ MB


<a id="step6"></a>
### 2.4. Явные и неявные дубликаты в данных

- Изучите уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска. Проверьте, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- При необходимости проведите нормализацию данных с текстовыми значениями. Названия или жанры игр можно привести к нижнему регистру, а названия рейтинга — к верхнему.

In [22]:
for i in ['genre', 'platform', 'rating', 'year_of_release']:
    print(i.upper())
    print(df[i].unique())

GENRE
['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY']
PLATFORM
['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
RATING
['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']
YEAR_OF_RELEASE
<IntegerArray>
[2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010, 2013, 2004,
 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014, 1992, 1997, 1993, 1994,
 1982, 2016, 2003, 1986, 2000, <NA>, 1995, 1991, 1981, 1987, 1980, 1983]
Length: 38, dtype: Int64


- После того как нормализуете данные и устраните неявные дубликаты, проверьте наличие явных дубликатов в данных.

In [23]:
df['genre'] = df['genre'].str.lower()
df['genre'].unique()

array(['sports', 'platform', 'racing', 'role-playing', 'puzzle', 'misc',
       'shooter', 'simulation', 'action', 'fighting', 'adventure',
       'strategy'], dtype=object)

In [24]:
df['rating'] = df['rating'].str.replace('K-A', 'E')
df['rating'].unique()

array(['E', nan, 'M', 'T', 'E10+', 'AO', 'EC', 'RP'], dtype=object)

При анализе категориальных столбцов были выявлены и устранены неявные дубликаты.

- **Жанры игр (`genre`)**:  
  Наблюдались повторяющиеся жанры в разных регистрах (например, `Action` и `ACTION`, `Sports` и `SPORTS`).  
  Для унификации все значения были приведены к нижнему регистру с помощью `.str.lower()`.  
  Это позволило избежать искусственного дробления категорий при группировке и анализе.

- **Рейтинг ESRB (`rating`)**:  
  Значение `'K-A'` (Kids to Adults) является устаревшим названием рейтинга `'E'` (Everyone), использовавшимся до 1998 года.  
  Поскольку по смыслу эти категории идентичны — обе указывают на то, что игра подходит для всех возрастов,  
  все вхождения `'K-A'` были заменены на `'E'`.  
  Это обеспечивает единообразие данных и корректность анализа по возрастным группам.

In [25]:
df.duplicated().sum()

241

In [26]:
df = df.drop_duplicates()

Были удалены явные дубликаты во всем датасете, их было 241.

In [27]:
deleted_rows = df_original.shape[0]- df.shape[0]

In [28]:
deleted_rows /df_original.shape[0]

0.014331210191082803

Количесиво пропусков в относительном значение 0,014.  

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  Int64  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       7590 non-null   float64
 10  rating           9949 non-null   object 
dtypes: Int64(1), float64(6), object(4)
memory usage: 1.5+ MB


- В процессе подготовки данных вы могли что-либо удалять, например строки с пропусками или ошибками, дубликаты и прочее. В этом случае посчитайте количество удалённых строк в абсолютном и относительном значениях.

**Промежуточный вывод:**

Название колонок датасета были приведены к стилю `snake_case`. 

Типы данных были приведены к нужному типу, а пропуски были обратаны.

В данных были выявлены явные неявные дубликаты. Неявные дубликаты сосредоточились в колонке genre, они обработаны и приведены к унифицированной форме. В остальных столбцах неявных дубликатов не обнаружилось. Обработанный датафрейм сохранен и имеет размер 16713 строк (по сравнению с 16956 строк от изначального датафрейма) после манипуляций.

---
<a id="step7"></a>
## 3. Фильтрация данных

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Отберите данные по этому показателю. Сохраните новый срез данных в отдельном датафрейме, например `df_actual`.

In [30]:
df_actual = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)].copy()

In [31]:
df_actual.shape

(12781, 11)

In [32]:
df.head(3)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E


Образован новый срез для работы с указанным периодом времени. 

---
<a id="step8"></a>
## 4. Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [33]:
df_actual['users_category'] = pd.cut(
    df_actual['user_score'], 
    bins=[0, 3, 8, 11], 
    labels=['Низкая оценка', 'Средняя оценка', 'Высокая оценка'], 
    right=False
)

- Разделите все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [34]:
df_actual['critics_category'] = pd.cut(
    df_actual['critic_score'], 
    bins=[0, 30, 80, 101],
    labels=["Низкая оценка", "Средняя оценка", "Высокая оценка"], 
    right=False
)

In [35]:
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,users_category,critics_category
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,Высокая оценка,Средняя оценка
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,Высокая оценка,Высокая оценка
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,Высокая оценка,Высокая оценка
6,New Super Mario Bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,Высокая оценка,Высокая оценка
7,Wii Play,Wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,Средняя оценка,Средняя оценка


- После категоризации данных проверьте результат: сгруппируйте данные по выделенным категориям и посчитайте количество игр в каждой категории.

In [36]:
df_actual.groupby('users_category')['name'].count()

users_category
Низкая оценка      116
Средняя оценка    4081
Высокая оценка    2286
Name: name, dtype: int64

In [37]:
df_actual.groupby('critics_category')['name'].count()

critics_category
Низкая оценка       55
Средняя оценка    5422
Высокая оценка    1692
Name: name, dtype: int64

- Выделите топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [42]:
df_actual.groupby('platform')['name'].count().nlargest(7)

platform
PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: name, dtype: int64

---
<a id="step9"></a>
## 5. Итоговый вывод

В конце напишите основной вывод и отразите, какую работу проделали. Не забудьте указать описание среза данных и новых полей, которые добавили в исходный датасет.

В рамках проекта выполнены все поставленные задачи по анализу и подготовке данных рынка видеоигр: <br>
Знакомство с данными – изучена структура датасета (16 956 строк, 11 столбцов), проверены типы данных, выявлены пропуски и аномалии.

Предобработка данных:
1. Стандартизация имён столбцов (snake_case)
2. Корректировка типов данных (year_of_release, eu_sales, jp_sales, user_score)
3. Обработка пропусков (удалены некритичные, сохранены важные для анализа)
4. Устранение дубликатов (явных и неявных)

Фильтрация данных – выделен период 2000–2013 для анализа современного этапа развития индустрии.

Категоризация данных – введены две новые аналитические переменные:
* users_category – категория пользовательской оценки (низкая/средняя/высокая)

* critic_category – категория экспертной оценки (низкая/средняя/высокая)

Анализ платформ – определён ТОП-7 платформ по количеству релизов.

**Ключевые инсайты**

Распределение оценок: <br>
*Пользовательские оценки*: преобладают средние (3–7), высоких (8–10) меньше, низких (0–2) минимальное количество<br>
*Экспертные оценки*: аналогичная картина с преобладанием средних оценок (30–79)

**ТОП-7 платформ (2000–2013):**

Абсолютными лидерами являются PS2 и DS с значительным отрывом от остальных<br>
В топ также входят: PS3, Wii, X360, PSP, PS